# Dashboard

In [ ]:

# Imports 

import pandas as pd


# Get the possible data venues
venuekeysfile = "venue-keys.csv"
dfVenueKeys = pd.read_csv(venuekeysfile)

dfDash = pd.DataFrame(columns = ["Venue","first_timestamp","last_timestamp","days_captured","last_voltage"])

for index, venueSensorDetails in dfVenueKeys.iterrows():
    try: 
        sensorMacOfSelection = venueSensorDetails['sensor_MAC']
        venueOfSelection = str(venueSensorDetails['venue_id'])
        dfTempDataSet = pd.read_csv('deviceData/'+ "venue_" + venueOfSelection + "_with_device_" + sensorMacOfSelection + '.csv' )
        dfTempDataSet['timestamp'] = pd.to_datetime(dfTempDataSet['timestamp'])
        dfTempDataSet['venue_id'] = venueSensorDetails['venue_id']
        dfTempDataSet = dfTempDataSet[(dfTempDataSet.temperature <= 100)]
        dfTempDataSet = dfTempDataSet[(dfTempDataSet.temperature > -10)] # eliminate rogue data
        #print("%s\t%s\t%s\t%d" %
        #    (venueOfSelection, dfTempDataSet.timestamp.min().strftime('%Y-%m-%d %H:%M'), dfTempDataSet.timestamp.max().strftime('%Y-%m-%d %H:%M'), len(dfTempDataSet.timestamp)/288))
        if (len(dfTempDataSet.timestamp)>0):
            first_time = dfTempDataSet.timestamp.min().strftime('%Y-%m-%d %H:%M')
            last_time = dfTempDataSet.timestamp.max().strftime('%Y-%m-%d %H:%M')
            days_captured = round(len(dfTempDataSet.timestamp.dropna())/288)
        else:
            first_time = "None"
            last_time = "None"
            days_captured = "None"
        try:
            volts = dfTempDataSet.voltage.dropna()
            last_voltage = volts.loc[volts != 1].iloc[-1]/10 

        except: 
            last_voltage = "None" # there aren't any good voltage readings
        dfDash.loc[len(dfDash)] = ([venueOfSelection, first_time, last_time, days_captured, last_voltage])
    except:
        print("Unknown dashboard error.")


In [ ]:

display(dfDash)